In [7]:
# importaing the required libraries
import os
import sys
base_path = r"C:\Users\KAI\Coding\ThinkOnward_challenge\thinkOnward_TSClassification"
data_path = r"\data\building-instinct-starter-notebook\Starter notebook"
sys.path.append(base_path+data_path)
path = r"C:\Users\KAI\Coding\ThinkOnward_challenge\thinkOnward_TSClassification\kai"
sys.path.append(base_path+path)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from tqdm import tqdm

from utils import (calculate_average_hourly_energy_consumption, train_model, get_pred, calculate_hierarchical_f1_score,
sample_submission_generator)
from preprocessing import Preprocessor

In [17]:
load_filepath_labels = os.path.join(os.getcwd(),'building-instinct-train-label', 'train_label.parquet')#path to the train label file
df_targets = pd.read_parquet(load_filepath_labels, engine='pyarrow')
target_columns = df_targets.columns

for type in tqdm(["monthly", "weekly", "daily"]):
    # train parquet files
    file_path = base_path + data_path + r"\building-instinct-train-data"
    df_features = Preprocessor.calculate_average_energy_consumption(file_path, type=type)
    df_features.sort_index(inplace=True)
    df = pd.concat([df_features, df_targets], axis=1)

    df.to_csv(f'{base_path}\preprocessed_data\{type}_train.csv')

    # test parquet files
    file_path = base_path + data_path + r"\building-instinct-test-data"
    df = Preprocessor.calculate_average_energy_consumption(file_path, type=type)
    df.sort_index(inplace=True)
    
    df.to_csv(f'{base_path}\preprocessed_data\{type}_test.csv')



100%|██████████| 1450/1450 [00:56<00:00, 25.76it/s]


In [19]:
df_targets.columns

Index(['building_stock_type', 'in.comstock_building_type_group_com',
       'in.heating_fuel_com', 'in.hvac_category_com',
       'in.number_of_stories_com', 'in.ownership_type_com', 'in.vintage_com',
       'in.wall_construction_type_com', 'in.tstat_clg_sp_f..f_com',
       'in.tstat_htg_sp_f..f_com', 'in.weekday_opening_time..hr_com',
       'in.weekday_operating_hours..hr_com', 'in.bedrooms_res',
       'in.cooling_setpoint_res', 'in.heating_setpoint_res',
       'in.geometry_building_type_recs_res', 'in.geometry_floor_area_res',
       'in.geometry_foundation_type_res', 'in.geometry_wall_type_res',
       'in.heating_fuel_res', 'in.income_res', 'in.roof_material_res',
       'in.tenure_res', 'in.vacancy_status_res', 'in.vintage_res'],
      dtype='object')

In [18]:
df = pd.read_csv(f'{base_path}\preprocessed_data\daily_train.csv')
print(df.columns)

Index(['bldg_id', '(1, 0)', '(1, 1)', '(1, 2)', '(1, 3)', '(1, 4)', '(1, 5)',
       '(1, 6)', '(1, 7)', '(1, 8)',
       ...
       'in.geometry_building_type_recs_res', 'in.geometry_floor_area_res',
       'in.geometry_foundation_type_res', 'in.geometry_wall_type_res',
       'in.heating_fuel_res', 'in.income_res', 'in.roof_material_res',
       'in.tenure_res', 'in.vacancy_status_res', 'in.vintage_res'],
      dtype='object', length=8786)


In [24]:
# target_columns = ['building_stock_type', 'in.comstock_building_type_group_com',
#        'in.heating_fuel_com', 'in.hvac_category_com',
#        'in.number_of_stories_com', 'in.ownership_type_com', 'in.vintage_com',
#        'in.wall_construction_type_com', 'in.tstat_clg_sp_f..f_com',
#        'in.tstat_htg_sp_f..f_com', 'in.weekday_opening_time..hr_com',
#        'in.weekday_operating_hours..hr_com', 'in.bedrooms_res',
#        'in.cooling_setpoint_res', 'in.heating_setpoint_res',
#        'in.geometry_building_type_recs_res', 'in.geometry_floor_area_res',
#        'in.geometry_foundation_type_res', 'in.geometry_wall_type_res',
#        'in.heating_fuel_res', 'in.income_res', 'in.roof_material_res',
#        'in.tenure_res', 'in.vacancy_status_res', 'in.vintage_res']
load_filepath_labels = os.path.join(os.getcwd(),'building-instinct-train-label', 'train_label.parquet')#path to the train label file
df_targets = pd.read_parquet(load_filepath_labels, engine='pyarrow')
target_columns = df_targets.columns
for type in tqdm(["monthly", "weekly", "daily"]):
       #     df = pd.read_csv(f'{base_path}\preprocessed_data\{type}_train.csv')
       #     df_targets = df[target_columns]
       #     df_features = df[df.columns.difference(target_columns)].astype(float)
       file_path = base_path + data_path + r"\building-instinct-train-data"
       df_features = Preprocessor.calculate_average_energy_consumption(file_path, type=type)
       df_features.sort_index(inplace=True)

       X_train, X_test, y_train, y_test = train_test_split(df_features, df_targets, test_size=0.2, random_state=42)
       X_train = X_train.astype(float)
       X_test = X_test.astype(float)
       classifier_list = train_model(X = X_train, y = y_train)
       y_pred = get_pred(X = X_test, classifier_list = classifier_list, column_list = target_columns)
       f1_score = calculate_hierarchical_f1_score(y_test, y_pred)
       print(f"{type}-{f1_score}")

 33%|███▎      | 1/3 [05:42<11:24, 342.33s/it]

monthly-0.5390841394336244


 67%|██████▋   | 2/3 [12:31<06:21, 381.37s/it]

weekly-0.537573044442947


100%|██████████| 3/3 [23:53<00:00, 477.71s/it]

daily-0.5406388320166617


In [26]:
type = "monthly"
file_path = base_path + data_path + r"\building-instinct-train-data"
df_features = Preprocessor.calculate_average_energy_consumption(file_path, type=type)
df_features.sort_index(inplace=True)

X_train, X_test, y_train, y_test = train_test_split(df_features, df_targets, test_size=0.2, random_state=42)
X_train = X_train.astype(float)
X_test = X_test.astype(float)

100%|██████████| 7200/7200 [04:40<00:00, 25.64it/s]


In [29]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score


X = X_train.copy()
y= y_train.copy()

# Define column transformers for commercial and residential buildings
transformer_commercial = ColumnTransformer([
    ('scaler', StandardScaler(), X.columns),
    ('encoder', OneHotEncoder(), [])
])

transformer_residential = ColumnTransformer([
    ('scaler', StandardScaler(), X.columns),
    ('encoder', OneHotEncoder(), [])
])

# Filter features and targets for commercial and residential buildings
X_commercial = X[y['building_stock_type'] == 'commercial']
X_residential = X[y['building_stock_type'] == 'residential']
y_commercial = y[y['building_stock_type'] == 'commercial'].filter(like='_com')
y_residential = y[y['building_stock_type'] == 'residential'].filter(like='_res')

# Train classifier to predict 'building_stock_type'
classifier_type = Pipeline([
    ('preprocessor', ColumnTransformer([
        ('scaler', StandardScaler(), X.columns),
        ('encoder', OneHotEncoder(), [])
    ])),
    ('classifier', RandomForestClassifier(random_state=42))
])
classifier_type.fit(X, y['building_stock_type'])

hp_params = {"n_estimators":2000, "criterion":'gini', "max_depth":None, "min_samples_split":2, "min_samples_leaf":1, "min_weight_fraction_leaf":0.0,
            "max_features":'sqrt', "max_leaf_nodes":None, "min_impurity_decrease":0.0, "bootstrap":True, "oob_score":False, "n_jobs":None, "random_state":42, 
            "verbose":0, "warm_start":False, "class_weight":None, "ccp_alpha":0.0, "max_samples":None, "monotonic_cst":None}

# Train separate classifiers for commercial and residential buildings
classifier_commercial = Pipeline([
    ('preprocessor', transformer_commercial),
    ('classifier', RandomForestClassifier(**hp_params))
])

classifier_residential = Pipeline([
    ('preprocessor', transformer_residential),
    ('classifier', RandomForestClassifier(**hp_params))
])

# Train models
classifier_commercial.fit(X_commercial, y_commercial)
classifier_residential.fit(X_residential, y_residential)
classifier_list = [classifier_type, classifier_residential, classifier_commercial]

In [30]:
y_pred = get_pred(X = X_test, classifier_list = classifier_list, column_list = target_columns)
f1_score = calculate_hierarchical_f1_score(y_test, y_pred)
print(f"{type}-{f1_score}")

monthly-0.5152681689480081
